This is the neural network based on DenseNet and LearnableResizer architectures used for the classification improvement on the dataset from Skrabanek and Zahradnikova Jr., 2019

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate, LeakyReLU, Add,Resizing, Rescaling
import tensorflow.keras.backend as K
import numpy as np

#the model architecture

def conv_block(x, filters, kernel_size, strides, activation=LeakyReLU(0.2)):
    x = Conv2D(filters, kernel_size, strides, padding="same", use_bias=False)(x)
    x = BatchNormalization()(x)
    if activation:
        x = activation(x)
    return x


def res_block(x):
    inputs = x
    x = conv_block(x, 16, 3, 1)
    x = conv_block(x, 16, 3, 1, activation=None)
    return Add()([inputs, x])


def LearnableResizer(x,filters=16, num_res_blocks=1, interpolation="bilinear"):

    naive_resize = Resizing(256,256, interpolation=interpolation)(x)

    x = Conv2D(filters=filters, kernel_size=7, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=filters, kernel_size=1, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = BatchNormalization()(x)

    bottleneck = Resizing(256,256, interpolation=interpolation)(x)

    for _ in range(num_res_blocks):
        x = res_block(bottleneck)

    x = Conv2D(
        filters=filters, kernel_size=3, strides=1, padding="same", use_bias=False
    )(x)
    x = BatchNormalization()(x)

    x = Add()([bottleneck, x])

    # Final resized image.
    x = Conv2D(filters=3, kernel_size=7, strides=1, padding="same")(x)
    final_resize = Add()([naive_resize, x])

    return final_resize

def DenseNetNew(k, comp_f):
        #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition, filters):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x,m):
        
        x = bn_rl_conv(x, m )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    inputs = Input ((None,None,1))
    x = Rescaling(scale=1.0 / 255)(inputs)
    x = LearnableResizer(x)
    x = Conv2D(2*k, 7, strides = 2, padding = 'same')(x)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    m = 2*k
    
    for i,repetition in enumerate([6,9,12,15,18]):
        d = dense_block(x, repetition, k)
        m = np.floor((m + repetition*k)*comp_f)
        if i != 5:
            x = transition_layer(d,m)
    x = GlobalAveragePooling2D()(d)
    output = Dense(6, activation = 'softmax', kernel_regularizer="l2")(x)
    
    model = Model(inputs, output)
    return model

modelnew = DenseNetNew(15,0.5)
modelnew.summary()

In [ ]:
#example of model training
train_ds = tf.keras.utils.image_dataset_from_directory("T-1024-B-Small", batch_size = 6, shuffle = True, image_size = (1024,1024),color_mode = "grayscale",label_mode="categorical")
test_ds = tf.keras.utils.image_dataset_from_directory("E-1024-A-Small", batch_size = 6, shuffle = False, image_size = (1024,1024),color_mode = "grayscale",label_mode="categorical")
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-4, beta_1 = 0.95, beta_2 = 0.999)
modelnew.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="RescaledModel_{epoch}.h5",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    verbose = 1,
    save_best_only=True)
# Model weights are saved at the end of every epoch, if it's the best seen so far.
modelnew.fit(train_ds,epochs=5, callbacks=[model_checkpoint_callback], validation_data=test_ds)